In [ ]:
import pandas as pd

def read_csv_with_multiple_encodings(file_path, encodings=['utf-8', 'latin1', 'ISO-8859-1', 'cp1252']):
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            print(f"File read successfully with encoding: {encoding}")
            return df
        except UnicodeDecodeError:
            print(f"Failed to read file with encoding: {encoding}")
    raise ValueError("Unable to read the file with the provided encodings")

file_path = "E://files_pandas//datos_abiertos//IRS_localidad_2020.csv"
df_irs = read_csv_with_multiple_encodings(file_path)


DATA FRAME ANALFABETISMO

In [ ]:
df_irs_hgo = df_irs[df_irs['ent'] == 'Hidalgo']
columnas = ['cve_loc','mun','loc','i_analf']
df_analf = df_irs_hgo[ columnas]
df_analf = df_analf.sort_values(by='i_analf', ascending=False)
df_geoloc = pd.read_csv('E://files_pandas//sisver//AGEEML_202410311056515.csv') # csv con las coordenadas de las localidades 
df_analf = df_analf.merge(df_geoloc[['NOM_LOC', 'LAT_DECIMAL', 'LON_DECIMAL']], 
                          left_on='loc', right_on='NOM_LOC', how='left')# merge de los dos df para obtener las coordenadas 

df_analf.rename(columns={'LAT_DECIMAL': 'lat', 'LON_DECIMAL': 'lon'}, inplace=True)#renombrar columnas 
df_analf = df_analf.dropna(subset=['lat', 'lon'])#quitar los missing values para poder mapear


CONTORNO MAPA DEL ESTADO

In [ ]:
import geopandas as gpd
import folium
import pandas as pd
from folium.plugins import HeatMap
# Ruta al archivo SHP
shapefile_path = 'E://mapas//13_hidalgo//conjunto_de_datos//13ent.shp'

# Leer el archivo SHP
shpg = gpd.read_file(shapefile_path)

# Filtrar por la columna NOMGEO que contiene 'HIDALGO'
shpg_hidalgo = shpg[shpg['NOMGEO'].str.contains('Hidalgo', case=False)]

# Crear el mapa base centrado en Hidalgo
m = folium.Map(location=[20.0911, -98.7624], zoom_start=8)  # Coordenadas aproximadas de Hidalgo

def estilo_poligono(feature): 
    return { 
        'color': 'black', # Color de borde 
        'weight': 2, # Grosor del borde 
        'fillOpacity': 0.0 # Transparencia del relleno 
        }

folium.GeoJson(shpg_hidalgo, style_function=estilo_poligono).add_to(m)

heat_data = [[row['lat'], row['lon'], row['i_analf']] for index, row in df_analf.iterrows()]

# Agregar el mapa de calor al mapa base
HeatMap(heat_data).add_to(m)
m

m.save('E://files_pandas//datos_abiertos//mapas//mapa_calor_analfabetismo_hidalgo.html')